In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from poly_utils import get_markets, PLATFORM_WALLETS

pl.Config.set_tbl_rows(25)
pl.Config.set_tbl_cols(-1)  # Show all columns
cfg = pl.Config.set_tbl_width_chars(1000)  # Wider display


In [2]:
markets_df = get_markets()

Loaded 142247 markets from markets_partitioned
Combined total: 142239 unique markets (sorted by createdAt)


In [4]:
df = pl.scan_parquet("processed/trades/**/*.parquet").with_columns(
     pl.col("timestamp").str.to_datetime()
)

In [5]:
USERS = {
    'domah': '0x9d84ce0306f8551e02efef1680475fc0f1dc1344',
    '50pence': '0x3cf3e8d5427aed066a7a5926980600f6c3cf87b3',
    'fhantom': '0x6356fb47642a028bc09df92023c35a21a0b41885',
    'car': '0x7c3db723f1d4d8cb9c550095203b686cb11e5c6b',
    'theo4': '0x56687bf447db6ffa42ffe2204a05edaa20f55839'
}

In [7]:
# a easy hack that does a good enough job of determining redemptions

df = df.with_columns(
    pl.col("price")
      .sort_by("timestamp")          # ensure we use the latest trade within each group
      .last()
      .over(["market_id", "nonusdc_side"])
      .alias("last_price")
)

df = df.with_columns(
    last_price = (
        pl.when(pl.col("last_price") > 0.98).then(pl.lit(1.0))
         .when(pl.col("last_price") < 0.02).then(pl.lit(0.0))
         .otherwise(pl.col("last_price"))
    )
)


### This is how polymarket generates its events and this is how you get all trades for a given user. Even if it looks like we are only getting data where the user is a maker, that is not how it works on the contract level. maker shows trades from that users pov including price

In [8]:
trader_df = df.filter((pl.col("maker") == USERS['domah'])).collect()

In [9]:
trader_df = trader_df.select(['timestamp', 'market_id', 'maker', 'taker', 'maker_direction', 'nonusdc_side', 'price', 'token_amount', 'usd_amount', 'transactionHash', 'last_price'])
trader_df = trader_df.rename({'maker_direction': 'direction', 'nonusdc_side': 'side'})

TypeError: LazyFrame is not subscriptable (aside from slicing)

Use `select()` or `filter()` instead.

In [13]:
trader_df = (
    trader_df
    .group_by(["market_id", "side"])
    .agg(
        # USD volumes
        (pl.when(pl.col("direction") == "BUY").then(pl.col("usd_amount")).otherwise(0.0)).sum().alias("buy_usd"),
        (pl.when(pl.col("direction") == "SELL").then(pl.col("usd_amount")).otherwise(0.0)).sum().alias("sell_usd"),

        # Token volumes
        (pl.when(pl.col("direction") == "BUY").then(pl.col("token_amount")).otherwise(0.0)).sum().alias("buy_tokens"),
        (pl.when(pl.col("direction") == "SELL").then(pl.col("token_amount")).otherwise(0.0)).sum().alias("sell_tokens"),

        # Notionals for VWAPs
        (pl.when(pl.col("direction") == "BUY").then(pl.col("price") * pl.col("token_amount")).otherwise(0.0)).sum().alias("buy_notional"),
        (pl.when(pl.col("direction") == "SELL").then(pl.col("price") * pl.col("token_amount")).otherwise(0.0)).sum().alias("sell_notional"),

        pl.len().alias("trades"),
        pl.col("last_price").last().alias("last_price"),
    )
    .with_columns(
        (pl.col("sell_usd") - pl.col("buy_usd")).alias("cash_pnl_usd"),
        (pl.col("buy_tokens") - pl.col("sell_tokens")).alias("inventory_tokens"),
    )
    .with_columns(
        (pl.col("inventory_tokens") * pl.col("last_price")).alias("unrealized_usd"),
    )
    .with_columns(
        (pl.col("cash_pnl_usd") + pl.col("unrealized_usd")).alias("total_pnl_usd"),
    )
).collect()

In [ ]:
trader_df['total_pnl_usd'].sum() #domers total PnL. Checks out within 1% in the UI

2193037.3678080263